In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import svm
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.linear_model import Lasso,LinearRegression,LogisticRegression,ElasticNet
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.tree import DecisionTreeClassifier,BaseDecisionTree,ExtraTreeClassifier
from sklearn.ensemble import BaggingClassifier,StackingClassifier,RandomForestClassifier,VotingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder

import warnings
warnings.filterwarnings("ignore")

le = LabelEncoder()
ohe = OneHotEncoder()
sc = StandardScaler()

rfc =  RandomForestClassifier(n_estimators=100)
gnb = GaussianNB()
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier()
extratree = ExtraTreeClassifier()
knnc = KNeighborsClassifier()
svrc = svm.SVR()
lasso = Lasso()

In [2]:
df = pd.read_csv('train.csv')
df.tail(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y
613,LP002990,Female,No,0,Graduate,Yes,4583,0.0,133.0,360.0,0.0,Semiurban,N


In [3]:
df.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [4]:
df.Credit_History.fillna(0.0,inplace=True)
df.Credit_History.value_counts()

1.0    475
0.0    139
Name: Credit_History, dtype: int64

In [5]:
df.LoanAmount.fillna(df.LoanAmount.mean(),inplace=True)

In [6]:
df.Loan_Amount_Term.fillna(df.Loan_Amount_Term.mean(),inplace=True)
#df.Loan_Amount_Term.value_counts()

In [7]:
df.Self_Employed.fillna('No',inplace=True)
df.Self_Employed.value_counts()
#df.Self_Employed.replace({'No':1,'Yes':1})

No     532
Yes     82
Name: Self_Employed, dtype: int64

In [8]:
df.Dependents.fillna('0',inplace=True)
df.Dependents.value_counts()
#df.Dependents.replace({'0':0,'1':1,'2':2,'3+':3})

0     360
1     102
2     101
3+     51
Name: Dependents, dtype: int64

In [9]:
df.Gender.fillna('Female',inplace=True)
df.Gender.value_counts()
# df.Gender.replace({'Male':1,'Female':0})

Male      489
Female    125
Name: Gender, dtype: int64

In [10]:
df.Married.fillna('No',inplace=True)
df.Married.value_counts()
#df.Married.replace({'Yes':1,'No':0})

Yes    398
No     216
Name: Married, dtype: int64

In [11]:
df.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [12]:
df.Education.value_counts()
# df.Education.replace({'Graduate':1,'Not Graduate':0})

Graduate        480
Not Graduate    134
Name: Education, dtype: int64

In [13]:
df.Property_Area.value_counts()
# df.Property_Area.replace({'Semiurban':1,'Urban':2,'Rural':3})

Semiurban    233
Urban        202
Rural        179
Name: Property_Area, dtype: int64

In [14]:
print(192/422)
df.Loan_Status.value_counts()
# df.Loan_Status.replace({'Yes':1,'No':0})

0.4549763033175355


Y    422
N    192
Name: Loan_Status, dtype: int64

In [15]:
df.Self_Employed.replace({'No':1,'Yes':1},inplace=True)
df.Dependents.replace({'0':0,'1':1,'2':2,'3+':3},inplace=True)
df.Gender.replace({'Male':1,'Female':0},inplace=True)
df.Married.replace({'Yes':1,'No':0},inplace=True)
df.Loan_Status.replace({'Yes':1,'No':0},inplace=True)
df.Education.replace({'Graduate':1,'Not Graduate':0},inplace=True)
df.Property_Area.replace({'Semiurban':1,'Urban':2,'Rural':3},inplace=True)

In [16]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1,0,0,1,1,5849,0.0,146.412162,360.0,1.0,2,Y
1,LP001003,1,1,1,1,1,4583,1508.0,128.000000,360.0,1.0,3,N
2,LP001005,1,1,0,1,1,3000,0.0,66.000000,360.0,1.0,2,Y
3,LP001006,1,1,0,0,1,2583,2358.0,120.000000,360.0,1.0,2,Y
4,LP001008,1,0,0,1,1,6000,0.0,141.000000,360.0,1.0,2,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,0,0,0,1,1,2900,0.0,71.000000,360.0,1.0,3,Y
610,LP002979,1,1,3,1,1,4106,0.0,40.000000,180.0,1.0,3,Y
611,LP002983,1,1,1,1,1,8072,240.0,253.000000,360.0,1.0,2,Y
612,LP002984,1,1,2,1,1,7583,0.0,187.000000,360.0,1.0,2,Y


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             614 non-null    int64  
 2   Married            614 non-null    int64  
 3   Dependents         614 non-null    int64  
 4   Education          614 non-null    int64  
 5   Self_Employed      614 non-null    int64  
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         614 non-null    float64
 9   Loan_Amount_Term   614 non-null    float64
 10  Credit_History     614 non-null    float64
 11  Property_Area      614 non-null    int64  
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(7), object(2)
memory usage: 62.5+ KB


#sns.pairplot(df,hue='Loan_Status')
<img src='pairplot.png'/>

In [18]:
x = df.drop(['Loan_ID','Loan_Status'],axis='columns')
y = df.Loan_Status

In [19]:
x

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,1,0,0,1,1,5849,0.0,146.412162,360.0,1.0,2
1,1,1,1,1,1,4583,1508.0,128.000000,360.0,1.0,3
2,1,1,0,1,1,3000,0.0,66.000000,360.0,1.0,2
3,1,1,0,0,1,2583,2358.0,120.000000,360.0,1.0,2
4,1,0,0,1,1,6000,0.0,141.000000,360.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,1,1,2900,0.0,71.000000,360.0,1.0,3
610,1,1,3,1,1,4106,0.0,40.000000,180.0,1.0,3
611,1,1,1,1,1,8072,240.0,253.000000,360.0,1.0,2
612,1,1,2,1,1,7583,0.0,187.000000,360.0,1.0,2


In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)
print(len(x_train))
print(len(x_test))

521
93


In [21]:
y_test[:10]

305    N
334    Y
603    Y
139    N
306    Y
16     Y
344    Y
242    Y
352    Y
595    Y
Name: Loan_Status, dtype: object

In [22]:
from sklearn.model_selection import cross_val_score

In [23]:
scores = cross_val_score(RandomForestClassifier(n_estimators=100),x,y,cv=5)
print(scores,'\n',scores.mean())

[0.77235772 0.67479675 0.72357724 0.71544715 0.7704918 ] 
 0.7313341330134613


In [24]:
algo = RandomForestClassifier(n_estimators=100)
algo.fit(x_train,y_train)
print(algo.score(x_test,y_test))
algo.predict(x_test[:10])

0.6666666666666666


array(['Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y'], dtype=object)

In [25]:
scores = cross_val_score(dtc,x,y,cv=5)
print(scores,'\n',scores.mean())

[0.67479675 0.64227642 0.71544715 0.66666667 0.68032787] 
 0.6759029721444756


In [45]:
algo = dtc
algo.fit(x_train,y_train)
print(algo.score(x_test,y_test))
algo.predict(x_test[:10])

0.6236559139784946


array(['N', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'N', 'N', 'N'], dtype=object)

In [27]:
scores = cross_val_score(gnb,x,y,cv=5)
print(scores,'\n',scores.mean())

[0.78861789 0.72357724 0.70731707 0.77235772 0.7704918 ] 
 0.7524723443955752


In [28]:
algo = gnb
algo.fit(x_train,y_train)
print(algo.score(x_test,y_test))
algo.predict(x_test[:10])

0.6989247311827957


array(['Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y'], dtype='<U1')

In [29]:
scores = cross_val_score(knnc,x,y,cv=5)
print(scores,'\n',scores.mean())

[0.63414634 0.61788618 0.62601626 0.58536585 0.6147541 ] 
 0.6156337465013995


In [30]:
algo = knnc
algo.fit(x_train,y_train)
print(algo.score(x_test,y_test))
algo.predict(x_test[:10])

0.6236559139784946


array(['N', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y'], dtype=object)

In [31]:
scores = cross_val_score(extratree,x,y,cv=5)
print(scores,'\n',scores.mean())

[0.68292683 0.70731707 0.64227642 0.63414634 0.63114754] 
 0.6595628415300546


In [32]:
algo = extratree
algo.fit(x_train,y_train)
print(algo.score(x_test,y_test))
algo.predict(x_test[:10])

0.6344086021505376


array(['Y', 'N', 'Y', 'Y', 'N', 'N', 'N', 'N', 'Y', 'Y'], dtype=object)

In [33]:
from sklearn.linear_model import RidgeClassifier
algo = RidgeClassifier()
algo.fit(x_train,y_train)
print(algo.score(x_test,y_test))
algo.predict(x_test[:10])

0.7096774193548387


array(['Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y'], dtype='<U1')

In [34]:
clf1 = gnb
clf2 = RandomForestClassifier(n_estimators=100)
clf3 = dtc

vote = VotingClassifier(estimators=[('gnb', clf1), ('rfc', clf2), ('dtc', clf3)], voting='hard')

vote.fit(x_train,y_train)
print(vote.score(x_test,y_test))
vote.predict(x_test[:10])

0.6451612903225806


array(['Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y'], dtype=object)

In [35]:
estimators = [('mnb',gnb),('rfc',RandomForestClassifier(n_estimators=100)),
              ('sgdc',dtc) ]

stack = StackingClassifier(estimators=estimators, final_estimator=gnb)#RandomForestClassifier(n_estimators=100))

stack.fit(x_train, y_train)
print(stack.score(x_test,y_test))
stack.predict(x_test[:10])

0.7204301075268817


array(['N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y'], dtype=object)

In [36]:
test = pd.read_csv('test.csv')
test

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777,113.0,360.0,1.0,Urban
363,LP002975,Male,Yes,0,Graduate,No,4158,709,115.0,360.0,1.0,Urban
364,LP002980,Male,No,0,Graduate,No,3250,1993,126.0,360.0,NaN,Semiurban
365,LP002986,Male,Yes,0,Graduate,No,5000,2393,158.0,360.0,1.0,Rural


In [37]:
test.Credit_History.fillna(0.0,inplace=True)
test.LoanAmount.fillna(df.LoanAmount.mean(),inplace=True)
test.Loan_Amount_Term.fillna(df.Loan_Amount_Term.mean(),inplace=True)
test.Self_Employed.fillna('No',inplace=True)
test.Dependents.fillna('0',inplace=True)
test.Gender.fillna('Female',inplace=True)
test.Married.fillna('No',inplace=True)

In [38]:
test.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [39]:
test.Self_Employed.replace({'No':1,'Yes':1},inplace=True)
test.Dependents.replace({'0':0,'1':1,'2':2,'3+':3},inplace=True)
test.Gender.replace({'Male':1,'Female':0},inplace=True)
test.Married.replace({'Yes':1,'No':0},inplace=True)
#test.Loan_Status.replace({'Yes':1,'No':0},inplace=True)
test.Education.replace({'Graduate':1,'Not Graduate':0},inplace=True)
test.Property_Area.replace({'Semiurban':1,'Urban':2,'Rural':3},inplace=True)

In [40]:
test

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,1,1,0,1,1,5720,0,110.0,360.0,1.0,2
1,LP001022,1,1,1,1,1,3076,1500,126.0,360.0,1.0,2
2,LP001031,1,1,2,1,1,5000,1800,208.0,360.0,1.0,2
3,LP001035,1,1,2,1,1,2340,2546,100.0,360.0,0.0,2
4,LP001051,1,0,0,0,1,3276,0,78.0,360.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,1,1,3,0,1,4009,1777,113.0,360.0,1.0,2
363,LP002975,1,1,0,1,1,4158,709,115.0,360.0,1.0,2
364,LP002980,1,0,0,1,1,3250,1993,126.0,360.0,0.0,1
365,LP002986,1,1,0,1,1,5000,2393,158.0,360.0,1.0,3


In [41]:
testt = test.drop('Loan_ID',axis='columns')
testt

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,1,1,0,1,1,5720,0,110.0,360.0,1.0,2
1,1,1,1,1,1,3076,1500,126.0,360.0,1.0,2
2,1,1,2,1,1,5000,1800,208.0,360.0,1.0,2
3,1,1,2,1,1,2340,2546,100.0,360.0,0.0,2
4,1,0,0,0,1,3276,0,78.0,360.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...
362,1,1,3,0,1,4009,1777,113.0,360.0,1.0,2
363,1,1,0,1,1,4158,709,115.0,360.0,1.0,2
364,1,0,0,1,1,3250,1993,126.0,360.0,0.0,1
365,1,1,0,1,1,5000,2393,158.0,360.0,1.0,3


In [46]:
algo.fit(x,y)
pre =pd.DataFrame({'Loan_ID':test.Loan_ID,'Loan_Status': algo.predict(testt)})
# pre.to_csv('loan_prediction_dtc.csv',index=False)
pre

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,N
3,LP001035,Y
4,LP001051,N
...,...,...
362,LP002971,Y
363,LP002975,Y
364,LP002980,Y
365,LP002986,Y


In [47]:
pre.Loan_Status.value_counts()

Y    242
N    125
Name: Loan_Status, dtype: int64

In [49]:
125/242 # dtc

0.5165289256198347